In [60]:
#!/usr/bin/env python
# coding: utf-8
import codecs
import csv
import re
import sys
import numpy as np
import pandas as pd
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from newspaper import Article
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
csv.field_size_limit(sys.maxsize)

9223372036854775807

In [61]:
# helper functions
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):

    text = text.lower().split()

    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
        
    return text

def get_body(url):
    a = Article(url, language='en')
    a.download()
    a.parse()
    return u''.join(a.text).encode('utf-8').strip()

In [62]:
#do this only once while in production
model = load_model('./model.h5')

read = pd.read_csv('./data/train.csv')
texts = [str(row[1]) + str(row[2]) for row in read.values]
labels = read[["type"]].values

read = pd.read_csv('./data/test.csv')
test_texts = [str(row[1]) + str(row[2]) for row in read.values]
test_ids = read.index.values

In [63]:
# get input from backend
_input = { "title": "Pakistan oil tanker inferno kills at least 123",
          "subtitle": "Pakistan oil tanker inferno kills at least 123",
          "description": "News",
          "website": "bbc.com",
          "poster": "BBC",
          "url": "http://www.breitbart.com/news/world-south-asia-40396036",
          "id": "world-south-asia-40396036" }

In [64]:
# clean input
text = [text_to_wordlist(_input['title'] + get_body(_input['url']))]
# tokenize input with train_data
tokenizer = Tokenizer(num_words=200000)
tokenizer.fit_on_texts(texts + test_texts + text)

# convert input to padded sequence
sequences = tokenizer.texts_to_sequences(text)
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=300)

In [65]:
# predict newsarticle validity
pred = model.predict([data], batch_size=1, verbose=0)

# output
output = {'id': _input['id'], 'prediction':int(round(pred[0][0])), 'confidence': pred[0][0]}

ValueError: Error when checking : expected input_1 to have shape (None, 500) but got array with shape (1, 300)

In [ ]:
output